In [1]:
from __future__ import unicode_literals, print_function, division

In [2]:
from io import open
import glob

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)


['data/names/Czech.txt', 'data/names/German.txt', 'data/names/Arabic.txt', 'data/names/Japanese.txt', 'data/names/Chinese.txt', 'data/names/Vietnamese.txt', 'data/names/Russian.txt', 'data/names/French.txt', 'data/names/Irish.txt', 'data/names/English.txt', 'data/names/Spanish.txt', 'data/names/Greek.txt', 'data/names/Italian.txt', 'data/names/Portuguese.txt', 'data/names/Scottish.txt', 'data/names/Dutch.txt', 'data/names/Korean.txt', 'data/names/Polish.txt']


In [3]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

Slusarski


In [4]:
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [5]:
for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [6]:
all_categories


['Czech',
 'German',
 'Arabic',
 'Japanese',
 'Chinese',
 'Vietnamese',
 'Russian',
 'French',
 'Irish',
 'English',
 'Spanish',
 'Greek',
 'Italian',
 'Portuguese',
 'Scottish',
 'Dutch',
 'Korean',
 'Polish']

In [7]:
category_lines['Czech']

['Abl',
 'Adsit',
 'Ajdrna',
 'Alt',
 'Antonowitsch',
 'Antonowitz',
 'Bacon',
 'Ballalatak',
 'Ballaltick',
 'Bartonova',
 'Bastl',
 'Baroch',
 'Benesch',
 'Betlach',
 'Biganska',
 'Bilek',
 'Blahut',
 'Blazek',
 'Blazek',
 'Blazejovsky',
 'Blecha',
 'Bleskan',
 'Blober',
 'Bock',
 'Bohac',
 'Bohunovsky',
 'Bolcar',
 'Borovka',
 'Borovski',
 'Borowski',
 'Borovsky',
 'Brabbery',
 'Brezovjak',
 'Brousil',
 'Bruckner',
 'Buchta',
 'Cablikova',
 'Camfrlova',
 'Cap',
 'Cerda',
 'Cermak',
 'Chermak',
 'Cermak',
 'Cernochova',
 'Cernohous',
 'Cerny',
 'Cerney',
 'Cerny',
 'Cerv',
 'Cervenka',
 'Chalupka',
 'Charlott',
 'Chemlik',
 'Chicken',
 'Chilar',
 'Chromy',
 'Cihak',
 'Clineburg',
 'Klineberg',
 'Cober',
 'Colling',
 'Cvacek',
 'Czabal',
 'Damell',
 'Demall',
 'Dehmel',
 'Dana',
 'Dejmal',
 'Dempko',
 'Demko',
 'Dinko',
 'Divoky',
 'Dolejsi',
 'Dolezal',
 'Doljs',
 'Dopita',
 'Drassal',
 'Driml',
 'Duyava',
 'Dvorak',
 'Dziadik',
 'Egr',
 'Entler',
 'Faltysek',
 'Faltejsek',
 'Fencl',

In [8]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
torch.Size([5, 1, 57])


In [9]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [10]:
n_letters

57

In [11]:
n_categories

18

In [12]:
input = Variable(lineToTensor('Albert'))
hidden = Variable(torch.zeros(1, n_hidden))

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.8359, -2.9183, -2.7858, -2.8683, -2.8951, -2.9869, -2.8819,
         -2.8700, -2.9283, -2.9163, -2.9253, -2.9432, -2.8717, -2.9386,
         -2.9041, -2.8238, -2.8996, -2.8536]])


In [13]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    line_tensor = Variable(lineToTensor(line))
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Spanish / line = Elizondo
category = Japanese / line = Nishimoto
category = Spanish / line = Obando
category = Dutch / line = Althuis
category = Arabic / line = Sarraf
category = Spanish / line = Busto
category = Italian / line = Sordi
category = Spanish / line = Arce
category = Russian / line = Binevich
category = Chinese / line = Hong


In [14]:
criterion = nn.NLLLoss()

In [27]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    print(output)
    print(output.size())
    print(category_tensor)
    print(category_tensor.size())
    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.data[0]

In [28]:
import time
import math

n_iters = 1
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

   
    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0
        

tensor([[-4.2613, -4.2731, -7.0222, -6.4710, -6.6671, -8.4939, -2.8248,
         -1.7289, -3.9250, -2.9994, -5.0146, -3.1225, -3.7795, -7.4290,
         -0.5509, -4.8784, -6.7022, -6.0821]])
torch.Size([1, 18])
tensor([ 0])
torch.Size([1])


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


In [18]:
def categoryFromOutput(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    category_i = top_i[0][0]
    return all_categories[category_i], category_i

In [25]:
a = torch.tensor([1],dtype=torch.int32)

In [26]:
print (a)

tensor([ 1], dtype=torch.int32)
